In [11]:
import numpy as np 
import pandas as pd
import os
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [12]:
path = Path.cwd().parent / 'data'
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        print(os.path.join(filename))

gender_submission.csv
test.csv
train.csv


In [13]:
df = pd.read_csv(path / 'train.csv')
df_test = pd.read_csv(path / 'test.csv')
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [14]:
df.dropna(subset=['Age'], inplace=True)
df_test['Age'] = df_test['Age'].fillna(np.mean(df['Age']))
df_test = df_test.fillna(0)

In [15]:
y_train_data = df['Survived']
features = df.drop(columns=['Cabin', 'Embarked', 'Ticket', 'Name', 'Survived'])
features_test = df_test.drop(columns=['Cabin', 'Embarked', 'Ticket', 'Name'])

X_train_data = pd.get_dummies(features)
X_test = pd.get_dummies(features_test)

In [16]:
X_train, X_val, y_train, y_val = train_test_split(X_train_data, y_train_data, test_size=0.2,random_state=42)
pd.DataFrame(X_train)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male
328,329,3,31.0,1,1,20.5250,True,False
73,74,3,26.0,1,0,14.4542,False,True
253,254,3,30.0,1,0,16.1000,False,True
719,720,3,33.0,0,0,7.7750,False,True
666,667,2,25.0,0,0,13.0000,False,True
...,...,...,...,...,...,...,...,...
92,93,1,46.0,1,0,61.1750,False,True
134,135,2,25.0,0,0,13.0000,False,True
337,338,1,41.0,0,0,134.5000,True,False
548,549,3,33.0,1,1,20.5250,False,True


In [17]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)
pd.DataFrame(X_train)

,0,1,2,3,4,5,6,7
0,0.368539,1.0,0.384267,0.2,0.166667,0.040062,1.0,0.0
1,0.082022,1.0,0.321438,0.2,0.000000,0.028213,0.0,1.0
2,0.284270,1.0,0.371701,0.2,0.000000,0.031425,0.0,1.0
3,0.807865,1.0,0.409399,0.0,0.000000,0.015176,0.0,1.0
4,0.748315,0.5,0.308872,0.0,0.000000,0.025374,0.0,1.0
...,...,...,...,...,...,...,...,...
566,0.103371,0.0,0.572757,0.2,0.000000,0.119406,0.0,1.0
567,0.150562,0.5,0.308872,0.0,0.000000,0.025374,0.0,1.0
568,0.378652,0.0,0.509927,0.0,0.000000,0.262527,1.0,0.0
569,0.615730,1.0,0.409399,0.2,0.166667,0.040062,0.0,1.0


In [18]:
svc = SVC(random_state=42,kernel = 'poly')
parameters = {'degree':range(0,10)}
clf = GridSearchCV(svc, parameters, cv=3, n_jobs=-1)
clf.fit(X_train, y_train)
sorted(clf.cv_results_.keys())
clf.best_params_

{'degree': 7}

In [19]:
res = pd.DataFrame(clf.cv_results_)
res.sort_values('mean_test_score', ascending = False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_degree,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
7,0.037670,0.010490,0.007635,0.002812,7,{'degree': 7},0.811518,0.826316,0.836842,0.824892,0.010387,1
9,0.100249,0.034233,0.002429,0.000310,9,{'degree': 9},0.795812,0.842105,0.831579,0.823165,0.019814,2
8,0.082179,0.019603,0.002684,0.000351,8,{'degree': 8},0.795812,0.836842,0.831579,0.821411,0.018229,3
4,0.011024,0.001675,0.002984,0.000198,4,{'degree': 4},0.806283,0.836842,0.821053,0.821392,0.012478,4
5,0.016893,0.004596,0.003640,0.000336,5,{'degree': 5},0.806283,0.821053,0.831579,0.819638,0.010375,5
6,0.030984,0.012064,0.005684,0.002187,6,{'degree': 6},0.795812,0.831579,0.821053,0.816148,0.015008,6
3,0.008367,0.000473,0.003677,0.001186,3,{'degree': 3},0.795812,0.810526,0.826316,0.810885,0.012456,7
2,0.009627,0.000759,0.003916,0.000553,2,{'degree': 2},0.774869,0.794737,0.810526,0.793377,0.014589,8
1,0.009230,0.000766,0.003922,0.001535,1,{'degree': 1},0.780105,0.794737,0.800000,0.791614,0.008417,9
0,0.013120,0.003725,0.009671,0.004027,0,{'degree': 0},0.591623,0.589474,0.589474,0.590190,0.001013,10


In [20]:
X_test = scaler.fit_transform(X_test)
output = pd.DataFrame({'PassengerId': df_test.PassengerId,'Survived': clf.predict(X_test)})

output.to_csv('submission.csv',index=0)
output

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
